In [1]:

try:
    import openfermion as of
    import openfermionpyscf as ofpyscf
except ImportError:
    print("Installing OpenFermion and OpenFermion-PySCF...")
    !pip install openfermion openfermionpyscf --quiet

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
import numpy as np
from scipy.sparse import linalg

import cirq
import openfermion as of
import openfermionpyscf as ofpyscf
from openfermion.chem import MolecularData
from openfermion.transforms import jordan_wigner
from openfermion.linalg import get_ground_state
import numpy
import scipy
import scipy.linalg
from openfermion.transforms import get_diagonal_coulomb_hamiltonian

In [112]:
import openfermion

# Load saved file for LiH.
diatomic_bond_length = 0.735
geometry = [("H", (0.0, 0.0, 0.0)), ("H", (0.0, 0.0, diatomic_bond_length))]
basis = "sto-3g"
multiplicity = 1
charge = 0

# Generate and populate instance of MolecularData.
molecule = MolecularData(geometry, basis, multiplicity, description="")
molecule.load()

# Get the Hamiltonian in an active space.
molecular_hamiltonian = molecule.get_molecular_hamiltonian(occupied_indices=None,active_indices=None)

# Map operator to fermions and qubits.
fermion_hamiltonian = of.get_fermion_operator(molecular_hamiltonian)

# Map to QubitOperator using the JWT
hamiltonian_jw = of.jordan_wigner(fermion_hamiltonian)

# Convert to Scipy sparse matrix
hamiltonian_jw_sparse = of.get_sparse_operator(hamiltonian_jw)

# Convert to DiagonalCoulombHamiltonian type.
hamiltonian = openfermion.transforms.get_diagonal_coulomb_hamiltonian(fermion_hamiltonian,ignore_incompatible_terms=True)
print(hamiltonian)
print(hamiltonian_jw_sparse)

  (0, 0)	(0.7559674441714287+0j)
  (1, 1)	(0.30766774806979075+0j)
  (2, 2)	(0.30766774806979064+0j)
  (3, 3)	(0.5644736841409372+0j)
  (12, 3)	(0.17900057606140662+0j)
  (4, 4)	(-0.5218855619854464+0j)
  (5, 5)	(-0.47845305583973213+0j)
  (6, 6)	(-0.2994524797783255+0j)
  (9, 6)	(-0.17900057606140662+0j)
  (7, 7)	(0.44908565854017335+0j)
  (8, 8)	(-0.5218855619854467+0j)
  (6, 9)	(-0.17900057606140662+0j)
  (9, 9)	(-0.2994524797783255+0j)
  (10, 10)	(-0.47845305583973213+0j)
  (11, 11)	(0.4490856585401733+0j)
  (3, 12)	(0.17900057606140662+0j)
  (12, 12)	(-1.1173490349902795+0j)
  (13, 13)	(-0.4031837505358063+0j)
  (14, 14)	(-0.4031837505358063+0j)
  (15, 15)	(1.0160871660914514+0j)


In [264]:
# Map to QubitOperator using the JWT
hamiltonian_jw = of.jordan_wigner(fermion_hamiltonian)

# Convert to Scipy sparse matrix
hamiltonian_jw_sparse = of.get_sparse_operator(hamiltonian_jw)

# Compute ground energy
eigs, _ = linalg.eigsh(hamiltonian_jw_sparse, k=1, which="SA")
ground_energy = eigs[0]

# Map to QubitOperator using the BK
hamiltonian_bk = of.bravyi_kitaev(fermion_hamiltonian)
hamiltonian_bk_sparse = of.get_sparse_operator(hamiltonian_bk)
eigs, _ = linalg.eigsh(hamiltonian_bk_sparse, k=1, which="SA")
ground_energ_bk = eigs[0]

print(f"Ground_energy using Jorden Wigner Transform: {ground_energy:.5f}")
print(f"Ground_energy using Bravyi Kitaev Transform: {ground_energ_bk:.5f}")

Ground_energy using Jorden Wigner Transform: -1.13619
Ground_energy using Bravyi Kitaev Transform: -1.13619


In [113]:
# Obtain the Bogoliubov transformation matrix.
#quadratic_hamiltonian = openfermion.ops.QuadraticHamiltonian(hamiltonian_jw_sparse)
quadratic_hamiltonian = openfermion.transforms.get_quadratic_hamiltonian(fermion_hamiltonian,
                                                                         chemical_potential=0.0,
                                                                         
                                                                         ignore_incompatible_terms=True)
print(quadratic_hamiltonian.ground_energy())

-2.6963379603455975


In [114]:
_, transformation_matrix, _ = quadratic_hamiltonian.diagonalizing_bogoliubov_transform()

In [115]:
occupied_orbitals = range(2)
n_qubits = openfermion.count_qubits(quadratic_hamiltonian)
qubits = cirq.LineQubit.range(n_qubits)
state_preparation_circuit = cirq.Circuit(openfermion.bogoliubov_transform(qubits, transformation_matrix, initial_state=occupied_orbitals))

In [116]:
# obtain Hamiltonian as an InteractionOperator
hamiltonian = ofpyscf.generate_molecular_hamiltonian(
    geometry, basis, multiplicity, charge
)

In [257]:
n_steps = 20
order = 0
time= 3
# Construct circuit
trotter_circuit = cirq.Circuit(openfermion.simulate_trotter(qubits, hamiltonian, time,
                                                            n_steps, 
                                                            order, algorithm=of.LOW_RANK))

In [263]:
%%time
simulator = cirq.Simulator()

# Obtain initial state vector as integer.
initial_state = sum(2 ** (n_qubits - 1 - i) for i in occupied_orbitals)

# Construct and simulate circuit using the swap network methodt
total_circuit= state_preparation_circuit+ trotter_circuit

result = simulator.simulate(total_circuit, initial_state=initial_state)
final_state = result.final_state_vector

print(f"Energy of hydrogen state obtained : {openfermion.expectation(hamiltonian_jw_sparse, final_state).real:.2f}")
 

Energy of hydrogen state obtained : -1.12
CPU times: user 267 ms, sys: 24 ms, total: 291 ms
Wall time: 283 ms
